# 0.0 BUSINESS ISSUE

## 0.1 Outputs:
- The output will be an answer for the business question: Does the new payment solution increases the revenue?

## 0.2 Process ( Step by step)
- What steps are necessary to get to the answers? 
1) Is the group comparables? Close % country and gender
- Define the delivery format 
The project output will be business question answer in jupyter notebook.

## 0.3 Input ( Data )
- Data Source: Csv downloaded in githublab with the following answers: uid, country, gender, money spent, qt of purchases, purchase date, group, device
- Tools: Python - Descriptive Statistics

## 0.4 packages import

In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.stats import api as sms
from scipy.stats import chi2_contingency
from scipy import stats
import matplotlib.pyplot as plt
import sweetviz as sv


In [28]:
aux0 = pd.crosstab( df1['country'], df1['group'] )
aux0.div(aux0.sum(axis=1), axis=0)

group,GRP A,GRP B
country,,
AUS,0.497542,0.502458
BRA,0.501499,0.498501
CAN,0.461335,0.538665
DEU,0.511364,0.488636
ESP,0.501551,0.498449
FRA,0.504222,0.495778
GBR,0.493414,0.506586
MEX,0.522069,0.477931
TUR,0.507492,0.492508


# 1.0 Experiment Desing

In [102]:
init_metric = 1902.86
end_metric = init_metric * ( 1 + 0.05 )
std_metric = 1398.89

effect_size = ( end_metric - init_metric ) / std_metric

sample_size = np.ceil(
    sms.tt_ind_solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.80
)).astype(int)

print( np.ceil( sample_size ).astype( int ))

3395


# 2.0 Data Preparation

## 2.1 Load Data

In [103]:
df0 = pd.read_csv("../data/ab_testing.csv")
df0.columns = df0.columns.str.lower()
print( df0.shape )
df0.head(3)

(45883, 8)


,uid,country,gender,spent,purchases,date,group,device
0,11115722,MEX,F,1595,5,2016-03-08,GRP B,I
1,11122053,USA,M,498,2,2017-07-14,GRP B,I
2,11128688,USA,F,2394,6,2017-09-17,GRP A,I


## 2.2 Descriptive Statistics

In [104]:
df2 = df0.copy()
df2 = df2[df2['country']=="BRA"]

In [105]:
df2['spent'].agg(['mean','std'])

mean    1902.860251
std     1398.886279
Name: spent, dtype: float64

In [106]:
print( df2['date'].min(), df2['date'].max() )

2014-03-04 2018-01-26


In [107]:
df2.groupby('group')[['uid']].count()

,uid
group,
GRP A,4518
GRP B,4491


In [108]:
# Check experimental unit
df2.groupby('uid')['group'].nunique().max()

1

In [109]:
#Check gender homogeneity
aux0 = pd.crosstab( df2['gender'], df2['group'] )
aux0.div(aux0.sum(axis=1), axis=0)

group,GRP A,GRP B
gender,,
F,0.500565,0.499435
M,0.502398,0.497602


In [110]:
#Check gender homogeneity
aux1 = pd.crosstab( df2['device'], df2['group'] )
aux1.div(aux1.sum(axis=1), axis=0)

group,GRP A,GRP B
device,,
A,0.491541,0.508459
I,0.511401,0.488599


In [111]:
df2.groupby( 'group' ).agg( {'country':"count",
"spent": ["min", "max", "mean", "std"],
"purchases": ['min','max','mean','std']})

country spent                                  purchases                \
        count   min    max         mean          std       min max      mean   
group                                                                          
GRP A    4518    99  10480  1899.792165  1413.074052         1  24  4.568172   
GRP B    4491    99  10478  1905.946782  1384.617019         1  23  4.585393   

                 
            std  
group            
GRP A  3.170057  
GRP B  3.132202

# 3.0 Hypotesis Test

## 3.1 Groups

In [112]:
sample_A = df2[df2['group'] == "GRP A"]['spent'].sample( n=sample_size, random_state=42)
sample_B = df2[df2['group'] == "GRP B"]['spent'].sample( n=sample_size, random_state=42)

#Metrics
print( "AVG Sample A:", sample_A.mean(), "AVG STD A:", sample_A.std() )
print( "AVG Sample B:", sample_B.mean(), "AVG STD B:", sample_B.std() )

AVG Sample A: 1908.559057437408 AVG STD A: 1406.420886628333
AVG Sample B: 1902.1045655375551 AVG STD B: 1383.9260171492604


## 3.2 Statistical Inference

In [114]:
t_test, p_value = stats.ttest_ind( sample_A, sample_B)
print( f"p-value= {p_value}")

p-value= 0.8488440918718094


## 4.0 Conclusion